In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

Calculate the self part of the Van Hove correlation function $G_s (r, t)$ at time 0 and $t^*$:
$$
  G_s (r, t_*) =  \frac{1}{N} \sum_{i=1}^{N} \delta [r - |r_i (t_*) - r_i (0)|] 
$$

In [ ]:
def dpart_func(frame0, frame_zv):
    
    dpart = []
    
    for part in frame0['particle']:
        
        if (part in frame_zv['particle'].unique()):
            mask0 = frame0['particle'] == part
            x0 = frame0[mask0]['x'].item()
            y0 = frame0[mask0]['y'].item()

            mask_zv = frame_zv['particle'] == part
            x_zv = frame_zv[mask_zv]['x'].item()
            y_zv = frame_zv[mask_zv]['y'].item()

#             dpart.append( abs(np.sqrt(x_zv*x_zv + y_zv*y_zv) - np.sqrt(x0*x0 + y0*y0)) )
            dpart.append( abs(np.sqrt((x_zv - x0)**2 + (y_zv - y0)**2)) )
    
    
    return dpart
    

In [ ]:
def aver_func(dpart, r, dr):
    
    G_s = np.zeros_like(r)
    
    N = len(dpart)
    
    for i, r_val in enumerate(r):
        
        count = np.sum((dpart >= r_val) & (dpart < r_val + dr))

        G_s[i] = count / N 
    
    return G_s

In [ ]:
file_name_prefix = '2_4000_6000'

In [ ]:
def read_data():
    
    file_path = f'data_out/backup_{file_name_prefix}/S6T34to18_t_corrected_{file_name_prefix}.csv'
    
    usecols = ['y', 'x', 'frame', 'particle']
    
    data = pd.read_csv(file_path, usecols=usecols)
    
    
    return data

In [ ]:
%%time


data = read_data()
# print(data.head())
data_frame0 = data[data['frame'] == 0]
max_frame = data['frame'].values[-1] + 1
frame_zv = 450 # t*


r = np.arange(0, 5, 0.0001)
dr = r[1] - r[0]


f0 = data[data['frame'] == 0]
f_zv = data[data['frame'] == frame_zv]


dpart = dpart_func(f0, f_zv)
Gs = aver_func(dpart, r, dr)


In [ ]:
print(Gs)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

plt.plot(r, Gs, label=r'$4\pi r^2 G_s(r, t)$')

ax.set(ylabel=r'$G_s$',
      xlabel='r')

plt.legend (fontsize=14)

In [ ]:
# # write to file
# DIR = f'data_out/backup_{file_name_prefix}/'

# file_PD = DIR + f'Probability_distribution_{file_name_prefix}.dat'

# data_out_dict = {"time[sec]": time, "mu[micron]": mu_micron}

# data_out = pd.DataFrame(data_out_dict)

# data_out.to_csv(file_PD, index=False)

In [ ]:
# # write to file
# DIR = f'data_out/backup_{file_name_prefix}/'

# file_Gs = DIR + f'G_s_{file_name_prefix}.dat'

# data_out_dict = {"r[px]": r, "G_s": G_s}

# data_out = pd.DataFrame(data_out_dict)

# data_out.to_csv(file_Gs, index=False)

In [ ]:
# Gaussian approximation of MSD

def Gauss_MSD(dpart, r):
    
    delta_r = np.mean( [x*x for x in dpart] )
    
    G0 = np.zeros_like(r)
    
    for i, r_val in enumerate(r):
#         G0[i] = np.sqrt( 1. / (2.*np.pi*delta_r) ) * np.exp( -r_val**2 / (2.*delta_r) )

        G0[i] = (1 / (np.pi * delta_r)) * np.exp( -r_val**2 / delta_r )
    
    
    return G0

In [ ]:
# convert to microns

px_to_micron = 0.06905

r_micron = [i * px_to_micron for i in r]

In [ ]:
G0 = Gauss_MSD(dpart, r)
print(G0)

In [ ]:
Gs_mult = np.zeros_like(r)
G0_mult = np.zeros_like(r)

for i, r_val in enumerate(r_micron):
    
    mult_s = 4*np.pi*r_val**2 #2*np.pi*r_val**2
    mult_0 = 4*np.pi*r_val**2 #2*np.pi*r_val**2 #2 * np.pi * r_val**2 #4. * np.pi*r_val*r_val
    
    Gs_mult[i] = mult_s * Gs[i]
    G0_mult[i] = mult_0 * G0[i]
    
print(Gs_mult)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

plt.plot(r_micron, Gs_mult, label=r'$4\pi r^2 G_s(r, t)$')
plt.plot(r_micron, G0_mult, label=r'$4\pi r^2 G_0(r, t)$')

ax.set(ylabel=r'$G_0, G_s$',
      xlabel='r')
# ax.set_xscale('log')
# ax.set_yscale('log')
plt.legend (fontsize=14)